In [13]:
isOpenset = False
existSavedFile = True

isLocalServer = True
sys_path_to_be_added = '/notebook/personal/ksuchoi216/FaceID_model_rebuild/'
config_path = './configs/config.json'

%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import os

def checkLocalServer(isLocalServer, sys_path_to_be_added=None):
    print(f'isLocalServer is {isLocalServer}')
    if isLocalServer is True:
        if sys_path_to_be_added not in sys.path:
            sys.path.insert(0, sys_path_to_be_added)
            os.chdir(sys_path_to_be_added)

checkLocalServer(isLocalServer, sys_path_to_be_added)

from utils import Config_Manager
cfgm = Config_Manager(config_path)
cfg = cfgm.get_cfg()
!pwd
!nvidia-smi

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
isLocalServer is True
/notebook/personal/ksuchoi216/FaceID_model_rebuild
Tue Oct 18 11:12:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:04:00.0 Off |                    0 |
| N/A   33C    P0    33W / 250W |   3237MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+------

In [14]:
from data_preprocessing import FolderDataset, DatasetFromNumpy
from data_preprocessing import buildDataLoaders, saveDataloaders

cfg["FolderDataset"]["folder_for_raw"] = cfg["folder_for_raw"]
folderDataset = FolderDataset(cfg["FolderDataset"])

folderDataset.setFilePath(
    './data',
    'extracted_np_org',
    'face',
    '.npy'
)

if not existSavedFile:
    print("loading numpy datasets...")
    emb, lb = folderDataset.createNumpyData()
    folderDataset.saveToNumpy(emb, lb)
emb, lb = folderDataset.loadFromNumpy()
print("creating a dataset from numpy datasets")



device is cuda:0
image size: 140
Loading faces from ./raw/photos_from_video
0 jhoh
1 jhongyoo
2 jhyoo
3 kschoi
path_emb: ./data/extracted_np_org/face_emb.npy
path_lb: ./data/extracted_np_org/face_lb.npy
loaded emb:(1157, 512) lb:(1157,)
creating a dataset from numpy datasets


In [15]:
from data_preprocessing import filterNumpy, splitNumpy, putTogetherData
from data_preprocessing import buildMultipleDataLoaders

if isOpenset:
    kkc = [0, 1, 2]
    kuc = [3]
    
    emb_kkc, lb_kkc = filterNumpy(
        data = emb,
        labels = lb,
        selected_labels = kkc
    )
    
    emb_kuc, lb_kuc = filterNumpy(
        data = emb,
        labels = lb,
        selected_labels = kuc
    )
    
    kkc_ratios = {'train': 0.7, 'val': 0.2, 'test': 0.1}
    data_kkc = splitNumpy(emb_kkc, lb_kkc, ratios=kkc_ratios)
    
    kuc_ratios = {'val': 0.7, 'test': 0.3}
    data_kuc = splitNumpy(emb_kuc, lb_kuc, ratios=kuc_ratios)
    
    total_data = putTogetherData(data_kkc, data_kuc)
    
    dataloaders = buildMultipleDataLoaders(total_data, batch_size=4)
    saveDataloaders(source_folder='data',
                    save_folder='dataloader_osr',
                    dataloaders=dataloaders)
    
else:
    dataloaders = buildDataLoaders(emb, lb,
                                   batch_size=4,
                                   ratio_train=0.7,
                                   ratio_val=0.2)
    saveDataloaders(source_folder='data',
                    save_folder='dataloader_org',
                    dataloaders=dataloaders)

dataset initial emb:(1157, 512) lb:(1157,)
dataset length: (1157) = tr (809) + val (231) + tt (117)
saved in ./data/dataloader_org/dataloader_train.pt
saved in ./data/dataloader_org/dataloader_val.pt
saved in ./data/dataloader_org/dataloader_test.pt
